# Getting and cleaning data

For the project, we retrive the dataset using the api for the open data of the City of Toronto. We clean duplicate identical rows that represent the same event and offence. Then, we examine the yearly counts and filter out old years with insufficient data. Finally, we save the dataset as a csv to be used in the analysis.

In [2]:
# Dependencies
import pandas as pd
import requests
from pprint import pprint

## Get data using API

The City of Toronto Open Data API has an upper limit for the number of records that we can get with each request. Therefore, we adapted an algorithm to retrieve the whole dataset using the limit and offset parameters (found in [this source](https://support.smartbear.com/qacomplete/docs/developer/api/rest/api/reference/paging.html)).

Based on the documentation for the API and dataset, we set the url for our requests.

We then get the dataset metadata to find the total number of records. We do this by setting the limit parameter to 0 so we only retrieve the metadata.

In [3]:
# Base url for all toronto open data
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources"
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
p = {"id": "major-crime-indicators"}
# get package resources
package = requests.get(url, params = p).json()
# Look at resources and find the one that is datastore_active
for resource in package["result"]["resources"]:
    # once we find the datastore_active resource, get the id
    if resource["datastore_active"]:
        resource_id = resource["id"]
# using that id and limit 0, get just the metadata first to find total items
# based on this algorithm:
# https://support.smartbear.com/qacomplete/docs/developer/api/rest/api/reference/paging.html
limit = 0

# build url for retrieve data
url = base_url + "/api/3/action/datastore_search"
# include resource id and limit in url parameters
p = {
    'id': resource_id,
    'limit': limit
}
# get metadata, and pprint it to find total items
metadata = requests.get(url, params = p).json()
pprint(metadata)

{'help': 'https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/help_show?name=datastore_search',
 'result': {'_links': {'next': '/api/3/action/datastore_search?id=af452875-cfdd-4596-a08a-7b93b65ea4f0&limit=0&offset=0',
                       'start': '/api/3/action/datastore_search?id=af452875-cfdd-4596-a08a-7b93b65ea4f0&limit=0'},
            'fields': [{'id': '_id', 'type': 'int'},
                       {'id': 'EVENT_UNIQUE_ID',
                        'info': {'notes': 'Offence Number'},
                        'type': 'text'},
                       {'id': 'REPORT_DATE',
                        'info': {'notes': 'Date Offence was Reported'},
                        'type': 'date'},
                       {'id': 'OCC_DATE',
                        'info': {'notes': 'Date of Offence'},
                        'type': 'date'},
                       {'id': 'REPORT_YEAR',
                        'info': {'notes': 'Year Offence was Reported'},
                        'type': 'f

Looking at the metadata, we find the total number of records within the item named 'result' and then 'total', so we save it to a variable. We also note that the actual data will be contained within the same 'result' item but in the subitem 'records'.

In [5]:
# get total items from metadata
total_items = metadata['result']['total']
total_items

323296

We are now ready to make the requests to retrieve the data. We set the limit parameter to the maximum that can be retrieved each time, based on the documentation (i.e. 32000). The offset parameter, which sets the starting record to retrieve will start at 0, since we want to start with the first record and increase by the limit amount each time we make a request. Each request will extend a list that contains the data and add the records retrieved in that request. We will repeat the request until the offset parameter is above the total number we found in the metadata. Then, we create a pandas dataframe using the final data list. At the end, we compare the total number that we got initially to the number of rows in the dataframe, to make sure they are equal.

In [6]:
# Update parameters to include both limit and offset
# From documentation, we know the max limit is 32000
# so we set, limit to that
p['limit'] = 32000

# offset will start at 0 and increase by 32000 each loop time
p['offset'] = 0

# create list to contain data
data_list = []

n = 1

# While offset is under total_items
while p['offset'] < total_items:
    # log number of request and parameters to be used
    print(f"request #{n} with parameters: {p}")
    # get data that is inside 'result' and 'records'
    data = requests.get(url, params = p).json()['result']['records']
    # extend data list
    data_list.extend(data)
    # To finish loop section, increase offset by limit
    p['offset'] += p['limit']
    n += 1

# Create dataframe with data
df = pd.DataFrame(data_list)
# print total length of dataframe, to confirm we got everything
print(f"\nThe total items in dataset is {total_items} and we got {len(df)}.")

# Check head of dataframe
df.head()


request #1 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 0}
request #2 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 32000}
request #3 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 64000}
request #4 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 96000}
request #5 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 128000}
request #6 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 160000}
request #7 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 192000}
request #8 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 224000}
request #9 with parameters: {'id': 'af452875-cfdd-4596-a08a-7b93b65ea4f0', 'limit': 32000, 'offset': 256000}
request #10 with parameters

Finally, we write the dataframe

In [7]:
# Write to csv
df.to_csv('resources/mci_all.csv', header = True, index = False)
